In [ ]:
try:
    import openmdao.api as om
except ImportError:
    !python -m pip install openmdao[notebooks]
    import openmdao.api as om

# DotProductComp

`DotProductComp` performs a dot product between two compatible inputs.  It may be vectorized to provide the result at one or more points simultaneously.

$$
    c_i = \bar{a}_i \cdot \bar{b}_i
$$

## DotProductComp Options

The default `vec_size` is 1, providing the dot product of $a$ and $b$ at a single
point.  The lengths of $a$ and $b$ are provided by option `length`.

Other options for DotProductComp allow the user to rename the input variables $a$ and $b$ and the output $c$, as well as specifying their units.

In [ ]:
om.show_options_table("openmdao.components.dot_product_comp.DotProductComp")

## DotProductComp Constructor

The call signature for the `DotProductComp` constructor is:

```{eval-rst}
    .. automethod:: openmdao.components.dot_product_comp.DotProductComp.__init__
        :noindex:
```

## DotProductComp Usage

There are often situations when numerous products need to be computed, essentially in parallel.
You can reduce the number of components required by having one `DotProductComp` perform multiple operations.
This is also convenient when the different operations have common inputs.

The ``add_product`` method is used to create additional products after instantiation.

```{eval-rst}
    .. automethod:: openmdao.components.dot_product_comp.DotProductComp.add_product
       :noindex:
```

## DotProductComp Example

In the following example DotProductComp is used to compute instantaneous power as the
dot product of force and velocity at 100 points simultaneously.  Note the use of
`a_name`, `b_name`, and `c_name` to assign names to the inputs and outputs.
Units are assigned using `a_units`, `b_units`, and `c_units`.
Note that no internal checks are performed to ensure that `c_units` are consistent
with `a_units` and `b_units`.


In [ ]:
import numpy as np

n = 24

p = om.Problem()

dp_comp = om.DotProductComp(vec_size=n, length=3, a_name='F', b_name='v', c_name='P',
                            a_units='N', b_units='m/s', c_units='W')

p.model.add_subsystem(name='dot_prod_comp', subsys=dp_comp,
                     promotes_inputs=[('F', 'force'), ('v', 'vel')])

p.setup()

p.set_val('force', np.random.rand(n, 3))
p.set_val('vel', np.random.rand(n, 3))

p.run_model()

# Verify the results against numpy.dot in a for loop.
expected = []
for i in range(n):
    a_i = p.get_val('force')[i, :]
    b_i = p.get_val('vel')[i, :]
    expected.append(np.dot(a_i, b_i))

    actual_i = p.get_val('dot_prod_comp.P')[i]
    rel_error = np.abs(expected[i] - actual_i)/actual_i
    assert rel_error < 1e-9, f"Relative error: {rel_error}"

print(p.get_val('dot_prod_comp.P', units='kW'))

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal

assert_near_equal(p.get_val('dot_prod_comp.P', units='kW'), np.array(expected)/1000.)